## Create Index

### imports

In [1]:
# Import required libraries  
import os  
from dotenv import load_dotenv  
from azure.core.credentials import AzureKeyCredential  
from azure.search.documents.indexes import SearchIndexClient  

from azure.search.documents.indexes.models import (  
    ExhaustiveKnnAlgorithmConfiguration,
    ExhaustiveKnnParameters,
    SearchIndex,  
    SearchField,  
    SearchFieldDataType,  
    SimpleField,  
    SearchableField,            
    HnswAlgorithmConfiguration,
    HnswParameters,  
    VectorSearch,    
    VectorSearchAlgorithmKind,
    VectorSearchProfile,    
    VectorSearchAlgorithmMetric,
    ComplexField
    
)  

  
# Configure environment variables  
load_dotenv()  

service_endpoint = os.getenv("NEW_AI_SEARCH_ENDPOINT_PROD") 
index_name = "aspen-conso-index-prd"
# index_name = os.getenv("BCFG_AZURE_AI_SEARCH_API_KEY") 
key = os.getenv("NEW_AI_SEARCH_API_KEY_PROD") 
model: str = "text-embedding-ada-002" 
credential = AzureKeyCredential(key)

### delete if exist index

In [2]:
from azure.core.exceptions import ResourceNotFoundError

# Create a search index client
index_client = SearchIndexClient(
    endpoint=service_endpoint, credential=credential)

try:
    # Check if the index exists
    index_client.get_index(index_name)
    # If exists, delete the index
    index_client.delete_index(index_name)
    print(f"Index '{index_name}' deleted.")
except ResourceNotFoundError:
    # If index does not exist, handle the exception
    print(f"Index '{index_name}' does not exist, no need to delete.")

Index 'aspen-conso-index-prd' deleted.


In [3]:
# Configure the vector search configuration  
vector_search = VectorSearch(
    algorithms=[
        HnswAlgorithmConfiguration(
            name="myHnsw",
            kind=VectorSearchAlgorithmKind.HNSW,
            parameters=HnswParameters(
                m=4,
                ef_construction=400,
                ef_search=500,
                metric=VectorSearchAlgorithmMetric.COSINE
            )
        ),
        ExhaustiveKnnAlgorithmConfiguration(
            name="myExhaustiveKnn",
            kind=VectorSearchAlgorithmKind.EXHAUSTIVE_KNN,
            parameters=ExhaustiveKnnParameters(
                metric=VectorSearchAlgorithmMetric.COSINE
            )
        )
    ],
    profiles=[
        VectorSearchProfile(
            name="myHnswProfile",
            algorithm_configuration_name="myHnsw",
        ),
        VectorSearchProfile(
            name="myExhaustiveKnnProfile",
            algorithm_configuration_name="myExhaustiveKnn",
        )
    ]
)


# create index
fields = [
    SimpleField(name="id", type=SearchFieldDataType.String, key=True, sortable=True, filterable=True, facetable=True),    
    SearchableField(name="content", type=SearchFieldDataType.String),
    SearchableField(name="file_location", type=SearchFieldDataType.String),     
    SearchableField(name="document_title", type=SearchFieldDataType.String, filterable=True),
    SearchableField(name="download_url", type=SearchFieldDataType.String),
    SearchableField(name="edit_url", type=SearchFieldDataType.String),
    SearchableField(name="category", type=SearchFieldDataType.String, filterable=True),
    SearchField(name="contentVector", type=SearchFieldDataType.Collection(SearchFieldDataType.Single),
                searchable=True, vector_search_dimensions=1536, vector_search_profile_name="myHnswProfile"),    
]

index = SearchIndex(name=index_name, fields=fields,
                    vector_search=vector_search, 
                    )
result = index_client.create_or_update_index(index)
print(f' {result.name} created')

 aspen-conso-index-prd created
